## Needed importations

In [1]:
import pandas as pd
import plotly.express as px
import folium as fl
from haversine import haversine

In [2]:
# Data uploading

df = pd.read_csv('train.csv')

In [3]:
# Cleaning the datas

df2 = df.copy()
df2 = df2[df2['Delivery_person_Age'] != 'NaN ']
df2 = df2[df2['City'] != 'NaN ']
df2 = df2[df2['multiple_deliveries'] != 'NaN ']
df2 = df2[df2['Road_traffic_density'] != 'NaN ']
df2 = df2[df2['Festival'] != 'NaN ']



- ## Converting the column Age from text to number

In [4]:
df2['Delivery_person_Age'] = df2['Delivery_person_Age'].astype(int)

- ## Converting the column Ratings from text to float

In [5]:
df2['Delivery_person_Ratings'] = df2['Delivery_person_Ratings'].astype(float)

- ## Converting the column order_date from text to date

In [6]:
df2['Order_Date'] = pd.to_datetime(df2['Order_Date'], format="%d-%m-%Y")

- ## Converting the column multiple_deliveries from text to number(int)

In [7]:
df2['multiple_deliveries'] = df2['multiple_deliveries'].astype(int)

- ## Removing the empty spaces into the strings/text/object

In [8]:
df2['ID'] = df2['ID'].str.strip() # It's needed to call the method 'str' before the 'strip'

In [9]:
df2['Road_traffic_density'] = df2['Road_traffic_density'].str.strip()

In [10]:
df2['Type_of_order'] = df2['Type_of_order'].str.strip()

In [11]:
df2['Type_of_vehicle'] = df2['Type_of_vehicle'].str.strip()

In [12]:
df2['City'] = df2['City'].str.strip()

> ## 1. Order quantity per day.

In [13]:
# Number of order grouped by date

df_aux = df2.loc[:, ['ID', 'Order_Date']].groupby('Order_Date').count().reset_index()
df_aux['ID'][0]

902

In [14]:
# Drawing a graphic of bar

px.bar(df_aux, x='Order_Date', y='ID')

> ## 2. Order quantity per Week.

In [15]:
# Creating a week column 

df2['week_of_year'] = df2['Order_Date'].dt.strftime('%U') #('%U') The counting of the days start from sunday

In [21]:
# Grouping the number of orders by week of the year

#df['week_of_year'] = df['week_of_year'].astype(int)
df_aux = df2[['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux.head(5)

,week_of_year,ID
0,06,1695
1,07,4941
2,09,4947
3,10,6700
4,11,6844


In [17]:
# Creating a graphic of lines

px.line(df_g_by_week, x='week_of_year', y='ID')

> ## 3. Order distribution by type of traffic.

In [18]:
df_aux = df2[['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()
df_aux['percentage'] = df_aux['ID']/df_aux['ID'].sum()
df_aux

,Road_traffic_density,ID,percentage
0,High,4105,0.099109
1,Jam,13174,0.318067
2,Low,14002,0.338057
3,Medium,10138,0.244767


In [22]:
# Making a graphic of pizza

px.pie(df_aux, values='percentage', names='Road_traffic_density')

> ## 4. Order's volume Comparison by city and type of traffic.

In [ ]:
df_aux = df2[['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
df_aux

In [ ]:
# Creating a bubble graphic

px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City')

>## 5. Order quantity per delivery person and per week

In [ ]:
# Grouping the quantity of orders by week of year
df_aux01 = df2[['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
# Grouping the quantity of orders by unique id of delivery person
df_aux02 = df2[['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()
# merging both dataframes
df_aux = pd.merge(df_aux01, df_aux02, how='inner')
# Quantity of delivery by each delivery person in a certain week of the year = qd_bydp_week
df_aux['qd_bydp_week'] = df_aux['ID']/df_aux['Delivery_person_ID']
df_aux

In [ ]:
# Creating a line graphic 
px.line(df_aux, x='week_of_year', y='qd_bydp_week')

>## 6. The central location of each city by type of traffic

In [ ]:
df_aux = df2[['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()
df_aux

In [ ]:
# Drawing a map 

_map = fl.Map()

for index, location in df_aux.iterrows():
    fl.Marker([location['Delivery_location_latitude'], 
               location['Delivery_location_longitude']]).add_to(_map)
_map

># The delivery person view

>## 1. The lowest and highest age of the delivery person.

In [ ]:
print(f'''The lowest age is {df2['Delivery_person_Age'].min()}
and the highest age is {df2['Delivery_person_Age'].max()}''')

>## 2. The worst and best vehicle condition

In [ ]:
print(f'''The worst  vehicle condition is {df2['Vehicle_condition'].min()}
and the best vehicle condition is {df2['Vehicle_condition'].max()}''')

>## 3. The average rating per delivery person.

In [ ]:
avg_ratings_dperson = df2[['Delivery_person_ID', 'Delivery_person_Ratings']].groupby('Delivery_person_ID').mean().reset_index()
avg_ratings_dperson

>## 4. The average rating and standard deviation per traffic type.

In [ ]:
std_avg_rating_by_trafic = (df2[['Delivery_person_Ratings', 'Road_traffic_density']]
            .groupby('Road_traffic_density').agg({'Delivery_person_Ratings':['mean', 'std']}))

std_avg_rating_by_trafic.columns = ['delivery_avg', 'delivery_std']
std_avg_rating_by_trafic = std_avg_rating_by_trafic.reset_index()
std_avg_rating_by_trafic

>## 5. The average rating and standard deviation per weather conditions.

In [ ]:
std_avg_rating_by_weatherconditions = (df2[['Delivery_person_Ratings', 'Weatherconditions']]
            .groupby('Weatherconditions').agg({'Delivery_person_Ratings':['mean', 'std']}))

std_avg_rating_by_weatherconditions.columns = ['delivery_avg', 'delivery_std']
std_avg_rating_by_weatherconditions = std_avg_rating_by_weatherconditions.reset_index()
std_avg_rating_by_weatherconditions

>## 6. The 10 fastest delivery person  per city.

In [ ]:
# Removing unnecessery information in the column 'Time_taken(min)'
# The information 'min' was removed of all lines
aux = df2['Time_taken(min)'].str.split(' ').apply(lambda x: x[1])

aux = aux.astype(int)
df2['Time_taken(min)'] = aux


In [ ]:
df_aux = (df2[['Delivery_person_ID', 'City', 'Time_taken(min)']]
          .groupby(['City', 'Delivery_person_ID']).mean()
          .reset_index())

# The 10 fastest delivery person in Metropolitian
fastest_Metropolitian = (df_aux[df_aux['City'] == 'Metropolitian']
                         .sort_values('Time_taken(min)').head(10))

# The 10 fastest delivery person in Urban
fastest_Urban = (df_aux[df_aux['City'] == 'Urban']
                         .sort_values('Time_taken(min)').head(10))

# The 10 fastest delivery person in Semi-Urban
fastest_Semi_Urban = (df_aux[df_aux['City'] == 'Semi-Urban']
                         .sort_values('Time_taken(min)').head(10))

In [ ]:
df3 = pd.concat([fastest_Metropolitian, fastest_Urban, fastest_Semi_Urban])
df3

>## 7. The 10 lowest delivery person  per city.

In [ ]:
# The 10 lowest delivery person in Metropolitian
lowest_Metropolitian = (df_aux[df_aux['City'] == 'Metropolitian']
                         .sort_values('Time_taken(min)', ascending=False).head(10))

# The 10 lowest delivery person in Urban
lowest_Urban = (df_aux[df_aux['City'] == 'Urban']
                         .sort_values('Time_taken(min)', ascending=False).head(10))

# The 10 lowest delivery person in Semi-Urban
lowest_Semi_Urban = (df_aux[df_aux['City'] == 'Semi-Urban']
                         .sort_values('Time_taken(min)', ascending=False).head(10))

In [ ]:
df4 = pd.concat([lowest_Metropolitian, lowest_Urban, lowest_Semi_Urban])
df4

># The Restaurants view

>## 1. The quantity of delivery person.


In [ ]:
print(f"The quantity of delivery person is {len(df2['Delivery_person_ID'].unique())}.")

>## 2. The average distance from the restaurant to the delivery location.


In [ ]:
# It will necessary to use the 'haversine' module 
cols = ['Restaurant_latitude', 'Restaurant_longitude', 
        'Delivery_location_latitude', 'Delivery_location_longitude']


df2['Distance(km)'] = (df2[cols].apply(lambda x: haversine((x['Restaurant_latitude'], x['Restaurant_longitude']), 
                        (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1))

In [ ]:
df2['Distance(km)'] = round(df2['Distance(km)'], 2)
df2[['Distance(km)']].head(5)

>## 3. The average time and standard deviation per delivery and city

In [ ]:
df_aux = (df2[['City', 'Time_taken(min)',]].groupby('City')
          .agg({'Time_taken(min)': ['mean', 'std']}))
df_aux.columns = ['Time_avg', 'Time_std']
df_aux = df_aux.reset_index()
df_aux

>## 5. The average time and standard deviation of delivery per city and kind of order

In [ ]:
df_aux = (df2[['City','Type_of_order', 'Time_taken(min)',]].groupby(['City', 'Type_of_order'])
          .agg({'Time_taken(min)': ['mean', 'std']}))
df_aux.columns = ['Time_avg', 'Time_std']
df_aux = df_aux.reset_index()
df_aux

>## 5. The average time and standard deviation of delivery per city and kind of traffic

In [ ]:
df_aux = (df2[['City','Road_traffic_density', 'Time_taken(min)',]].groupby(['City', 'Road_traffic_density'])
          .agg({'Time_taken(min)': ['mean', 'std']}))
df_aux.columns = ['Time_avg', 'Time_std']
df_aux = df_aux.reset_index()
df_aux

>## 6. The average time of delivery during Festivals

In [ ]:
df_aux = df2[['Festival', 'Time_taken(min)']].groupby('Festival').mean().reset_index()
df_aux
